In [1]:
import numpy as np
import pandas as pd

In [2]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc
from qapf import qapf
from qapf import cipw

___

# CIPWFULL normalized data

## Convert to CIPWFULL format

**I used an arbitrary version of the mineralogical data, not sure if this is the correct one** -->is a good one

In [3]:
mineralogy_original = pd.read_excel("../_CIPW/CIPW/AREA2/interpolated_data.xlsx", index_col=0)
# Only iterested in sample number and oxides
mineralogy = mineralogy_original[['SiO2', 'TiO2', 'Al2O3', 'Fe2O3','FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'P2O5', 'l.i.', 'oth']]
mineralogy.columns

Index(['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O',
       'K2O', 'P2O5', 'l.i.', 'oth'],
      dtype='object')

In [4]:
mineralogy = mineralogy.dropna()

In [5]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [6]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["l.i.", "oth"], axis=1))

In [7]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [8]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,P2O5
6777,71.723328,0.315689,14.884276,1.233404,1.935668,0.070970,0.771417,1.662899,3.564247,3.822293,0.015808
6778,71.837691,0.309795,14.848128,1.230966,1.905836,0.070226,0.749236,1.621748,3.567697,3.842724,0.015952
6779,71.842825,0.309633,14.845298,1.232037,1.904930,0.070288,0.747293,1.618594,3.568280,3.844905,0.015918
6780,71.849990,0.309356,14.841857,1.233096,1.903378,0.070328,0.745047,1.614810,3.568899,3.847355,0.015883
6781,71.866897,0.307965,14.841650,1.238391,1.892854,0.069794,0.742660,1.607396,3.568828,3.846713,0.016854


In [9]:
mineralogy

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,P2O5
6777,71.723328,0.315689,14.884276,1.233404,1.935668,0.070970,0.771417,1.662899,3.564247,3.822293,0.015808
6778,71.837691,0.309795,14.848128,1.230966,1.905836,0.070226,0.749236,1.621748,3.567697,3.842724,0.015952
6779,71.842825,0.309633,14.845298,1.232037,1.904930,0.070288,0.747293,1.618594,3.568280,3.844905,0.015918
6780,71.849990,0.309356,14.841857,1.233096,1.903378,0.070328,0.745047,1.614810,3.568899,3.847355,0.015883
6781,71.866897,0.307965,14.841650,1.238391,1.892854,0.069794,0.742660,1.607396,3.568828,3.846713,0.016854
...,...,...,...,...,...,...,...,...,...,...,...
261339,70.353303,0.347533,15.522520,1.199979,2.037145,0.056036,1.240068,2.306832,3.444279,3.374440,0.117865
261340,70.334569,0.348506,15.526417,1.201994,2.041417,0.056166,1.243493,2.312600,3.444053,3.372648,0.118137
261341,70.319076,0.349337,15.529503,1.204102,2.044933,0.056286,1.246084,2.317029,3.443957,3.371333,0.118360
261344,70.290849,0.351013,15.534308,1.210649,2.051258,0.056578,1.249306,2.322991,3.444343,3.369970,0.118737


In [10]:
mineralogy1 = mineralogy.iloc[0:45000]
mineralogy2 = mineralogy.iloc[45001:950000]

In [11]:
cipw.convert_to_CIPWFULL_format(mineralogy1, "../_CIPW/CIPW/AREA2/area2_1.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy2, "../_CIPW/CIPW/AREA2/area2_2.txt", dataset_name="Vistelius (1995)")

_____

## Extract results from CIPWFULL results file

**I only selected quartz, plagioclase and alkali feldspar here as they are of interest to us for the QAPF classification**  
**Feel free to extract addiotnal minerals if they would be of any interest to you**

**Not sure what to do with albite in the final assignments as by the book it should be accounted for as an alkali feldspar and thus be included in the sum of plagioclase but then the resulting plagioclase contents seem quite low.   
This is something you might look into although the method used here seems to be correct according to te resulting QAPF diagram.**

--> indeed when reading the paper of Streckeisen (and other sources), albite should be accounted for as an alkali feldspar, also saw that the datapoints were generally located more to the right side (plagioclase), maybe an overestimation of plagioclase?

--> I changed the settings below so that albite belongs to the alkali feldspars, Think this figure should be more correct according to the papers, but the cotent of plagioclase is indeed quite low. 

In [13]:
df_final1 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA2/area2_1_results_excel.txt")
df_final2 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA2/area2_2_results_excel.txt")

In [14]:
df_final = pd.concat([df_final1, df_final2])

In [15]:
df_final

,Q,P,K
SAMPLE,,,
6777,30.714441,35.287762,25.602160
6778,30.872406,35.108984,25.725803
6779,30.876396,35.097920,25.739176
6780,30.883366,35.084152,25.754179
6781,30.935518,35.040522,25.750338
...,...,...,...
261339,29.905159,36.901846,22.854250
261340,29.875734,36.926955,22.843477
261341,29.851463,36.946808,22.835602


## QAPF classification

In [16]:
df_final["QAPF"] = qapf.check_QAPF_region(df_final)

In [17]:
df_final["QAPF"].unique()

array(['monzo granite', 'granodiorite',
       'quartz monzodiorite\nquartz monzogabbro'], dtype=object)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.scatter_ternary(df_final, a="Q", b="K", c="P", color="QAPF")

fig.write_image(f"../_FIGURES/geomap/qapf_ternary/area1_QAPF.pdf")
fig.show()

In [18]:
df_final.to_excel("../_CIPW/CIPW/AREA2/qapf.xlsx", index=True)

-------

In [19]:
df_final.iloc[:, 3].value_counts().to_excel("../_CIPW/CIPW/AREA2/QAPF_counts.xlsx")

-----

In [20]:
QAPF_control = pd.read_excel("../_CIPW/CIPW/1_normalized.xlsx", index_col=0)
Area = pd.read_excel("../_INTERPOLATION/area_subdivided.xlsx", index_col=None, usecols = ['area', 'Y_UTM', 'X_UTM', 'ZoneNumber', 'ZoneLetter'])

In [21]:
QAPF_control = pd.concat([Area, QAPF_control], axis = 1)
QAPF_control = QAPF_control.dropna()
QAPF_control

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
1,-9.966614e+05,5.836313e+06,53.0,T,1.0,52.549507,17.912818,22.931202,monzo granite
2,-1.061190e+06,5.804252e+06,53.0,T,1.0,45.940433,19.086384,32.357653,monzo granite
3,-9.953983e+05,5.877750e+06,53.0,T,1.0,50.739986,6.408381,37.371890,syeno granite
4,-1.202844e+06,5.901332e+06,53.0,T,1.0,41.407116,23.232361,27.590113,monzo granite
5,-9.085210e+05,5.967408e+06,53.0,T,1.0,54.633030,17.945563,19.744849,monzo granite
...,...,...,...,...,...,...,...,...,...
4248,2.276617e+08,-4.667076e+08,53.0,T,5.0,16.144156,42.276924,25.207393,quartz monzonite
4249,1.898774e+08,-4.145875e+08,53.0,T,5.0,20.183025,45.679758,18.892849,granodiorite
4250,1.947005e+08,-4.390290e+08,53.0,T,5.0,11.994039,44.790421,23.532209,quartz monzodiorite\nquartz monzogabbro
4251,1.930846e+08,-4.358868e+08,53.0,T,5.0,15.151787,43.949506,22.328233,quartz monzodiorite\nquartz monzogabbro


In [24]:
QAPF_control_area2 = QAPF_control[QAPF_control['area'] ==2]
QAPF_control_area2.to_excel("../_CIPW/CIPW/AREA2/QAPF_control_points.xlsx", index=True)

In [25]:
QAPF_control_area2

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
1459,788062.538051,4.936585e+06,53.0,T,2.0,35.046036,33.988697,24.076958,monzo granite
1460,609204.390926,5.261750e+06,53.0,T,2.0,23.499948,39.281335,28.630845,monzo granite
1461,683161.686140,5.345172e+06,53.0,T,2.0,36.452862,33.156417,23.603618,monzo granite
1462,566277.769165,5.115580e+06,53.0,T,2.0,29.855810,38.642232,26.953392,monzo granite
1463,513561.171031,5.820370e+06,53.0,T,2.0,28.594946,35.774011,31.926062,monzo granite
...,...,...,...,...,...,...,...,...,...
2493,626698.329397,4.987292e+06,53.0,T,2.0,31.440487,31.189112,24.832439,monzo granite
2494,461509.372701,5.662951e+06,53.0,T,2.0,29.774464,35.409554,26.227575,monzo granite
2495,898558.158803,5.868297e+06,53.0,T,2.0,26.113008,37.450591,27.674605,monzo granite
2496,620259.749413,5.148324e+06,53.0,T,2.0,24.933060,33.943592,33.861921,monzo granite
